In [3]:
import pandas as pd
from datetime import datetime, timedelta

In [4]:
trimet_csv_file_url = 'https://drive.google.com/file/d/1G4MUbdUFeACgjGcg6dcwykBZn21zg1jO'
csv_file_id = trimet_csv_file_url.split('/')[-1]
df = pd.read_csv('https://drive.google.com/uc?export=download&id=' + csv_file_id)
df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259172515,259172517,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,12,0.7
1,259172515,259172517,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,12,0.8
2,259172515,259172517,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,12,0.8
3,259172515,259172517,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,12,0.7
4,259172515,259172517,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,12,0.8


In [5]:
df1 = df.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])
df1.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [12]:
df_using_usecols = pd.read_csv('https://drive.google.com/uc?export=download&id=' + csv_file_id, usecols=['EVENT_NO_TRIP', 'OPD_DATE', 'VEHICLE_ID', 'METERS', 'ACT_TIME', 'GPS_LONGITUDE', 'GPS_LATITUDE'])
df_using_usecols.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [13]:
def decodeTrimetDateCols(opd_date, act_time):
    opd_date_format = "%d%b%Y:%H:%M:%S"
    opd_datetime = datetime.strptime(opd_date, opd_date_format)
    total_opd_datetime = opd_datetime + timedelta(seconds=act_time)
    opd_timestamp = pd.Timestamp(total_opd_datetime, unit='s')
    return opd_timestamp

df_using_usecols['TIMESTAMP'] = df_using_usecols.apply(lambda each_row: decodeTrimetDateCols(each_row['OPD_DATE'], each_row['ACT_TIME']), axis=1)
df_using_usecols.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,2023-02-15 05:41:29


In [14]:
df2 = df_using_usecols.drop(columns=['OPD_DATE', 'ACT_TIME'])
df2.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29


In [20]:
df2['dMETERS'] = df2['METERS'].diff()
df2['dMETERS'] = df2['dMETERS'].fillna(0)
df2['dTIMESTAMP'] = df2['TIMESTAMP'].diff()
df2['dTIMESTAMP'].replace({pd.NaT: '0 days 00:00:00'}, inplace=True)
df2.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,dMETERS,dTIMESTAMP
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,0.0,0 days 00:00:00
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,8.0,0 days 00:00:05
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,9.0,0 days 00:00:05
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,16.0,0 days 00:00:05
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,39.0,0 days 00:00:05


In [24]:
def calculateSpeed(meters_diff, timestamp_diff):
    time_diff_in_sec = timestamp_diff.total_seconds()
    if time_diff_in_sec != 0:
        bus_speed = meters_diff / time_diff_in_sec
    else:
        bus_speed = 0.0
    return bus_speed

df2['SPEED'] = df2.apply(lambda each_row: calculateSpeed(each_row['dMETERS'], each_row['dTIMESTAMP']), axis=1)
df2.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,dMETERS,dTIMESTAMP,SPEED
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,0.0,0 days 00:00:00,0.0
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,8.0,0 days 00:00:05,1.6
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,9.0,0 days 00:00:05,1.8
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,16.0,0 days 00:00:05,3.2
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,39.0,0 days 00:00:05,7.8


In [25]:
df3 = df2.drop(columns=['dMETERS','dTIMESTAMP'])
df3.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,SPEED
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,0.0
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,1.6
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,1.8
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,3.2
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,7.8


In [31]:
df3['SPEED'].mask(df3['SPEED'] == 0.0).describe()

count    154.000000
mean       7.508785
std        4.263794
min        0.016667
25%        4.100000
50%        6.500000
75%       11.150000
max       17.400000
Name: SPEED, dtype: float64